In [24]:
import os
from PIL import Image, ImageDraw
import numpy as np
import math
import random
from tqdm import tqdm

In [51]:
random.seed(1714)

IMAGE_SET_NUM = 50
colour_1 = "yellow"
colour_2 = "blue"

# constants
boundary_width = 5
background_colour = "#808080"
yellow = "#fffe04"
blue = "#0003f9"
point_sep = 20
min_point_radius = 8
max_point_radius = 16
init_size = 512
mode = "RGB"

IMG_DIR = "../images/square"
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, colour_1), exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, colour_2), exist_ok=True)

In [63]:
def get_vertices(shape, center, radius):
    if shape == "circle":
        vertices = [
            (center[0] - radius, center[1] - radius),
            (center[0] + radius, center[1] + radius),
        ]
    elif shape == "triangle":
        vertices = [
            (center[0], center[1] - radius),
            (center[0] - radius, center[1] + radius),
            (center[0] + radius, center[1] + radius),
        ]
    elif shape == "square":
        vertices = [
            (center[0] - radius, center[1] - radius),
            (center[0] + radius, center[1] - radius),
            (center[0] + radius, center[1] + radius),
            (center[0] - radius, center[1] + radius),
        ]
    elif shape == "star":
        vertices = []
        for i in range(5):
            angle_deg = -90 + (i * 72)  # 72 degrees between each point
            angle_rad = math.radians(angle_deg)
            x = center[0] + radius * math.cos(angle_rad)
            y = center[1] + radius * math.sin(angle_rad)
            vertices.append((x, y))
            # Repeat for inner vertices
            angle_deg += 36
            angle_rad = math.radians(angle_deg)
            x = center[0] + (radius / 2) * math.cos(angle_rad)
            y = center[1] + (radius / 2) * math.sin(angle_rad)
            vertices.append((x, y))
    else:
        raise ValueError(f"Shape {shape} not implemented.")
    return vertices


def draw_shape(shape, radius, colour, it, img_dir, jitter=False):
    pixels_x = 512
    pixels_y = 512
    # Create a new image with grey background
    image = Image.new("RGB", (pixels_x, pixels_y), color="grey")

    col = yellow if colour == "yellow" else blue

    # Draw a blue point in the center
    draw = ImageDraw.Draw(image)
    # allow the center to move a bit
    x_space = int(pixels_x / 2 - radius) if jitter else 0
    y_space = int(pixels_y / 2 - radius) if jitter else 0
    dist_x = random.randint(-x_space, x_space)
    dist_y = random.randint(-y_space, y_space)
    center = (int(pixels_x / 2) + dist_x, int(pixels_y / 2) + dist_y)
    dist_from_center = np.sqrt(dist_x ** 2 + dist_y ** 2) 

    vertices = get_vertices(shape, center, radius)
    if shape == "circle":
        draw.ellipse(vertices, fill=col)
    else:    
        draw.polygon(vertices, fill=col)

    image.save(
        os.path.join(
            img_dir, colour, f"{shape}_{radius}_{int(dist_from_center)}_{it}.png"
        )
    )

In [61]:
def create_dirs(img_dir):
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(os.path.join(img_dir, colour_1), exist_ok=True)
    os.makedirs(os.path.join(img_dir, colour_2), exist_ok=True)


def create_set_of_images(
    shape,
    train_num,
    val_num,
    min_radius=40,
    max_radius=180,
    jitter=False,
    img_dir_tag="",
):
    IMG_DIR = f"../images/{img_dir_tag}{shape}"
    IMG_DIR_VAL = f"../images/{img_dir_tag}{shape}_val"
    create_dirs(IMG_DIR)
    create_dirs(IMG_DIR_VAL)

    print(f"Creating images for {shape} with {train_num} training images and {val_num} validation images")

    for i in tqdm(range(train_num)):
        for r in range(min_radius, max_radius):
            draw_shape(shape, r, "blue", i, IMG_DIR, jitter=jitter)
            draw_shape(shape, r, "yellow", i, IMG_DIR, jitter=jitter)

    for i in tqdm(range(val_num)):
        for r in range(min_radius, max_radius):
            draw_shape(shape, r, "blue", i, IMG_DIR_VAL, jitter=jitter)
            draw_shape(shape, r, "yellow", i, IMG_DIR_VAL, jitter=jitter)

In [56]:
create_set_of_images("square", 50, 10, 40, 180)
create_set_of_images("triangle", 50, 10, 40, 180)
create_set_of_images("circle", 50, 10, 40, 180)
create_set_of_images("star", 50, 10, 40, 180)

Creating images for square with 50 training images and 10 validation images


100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


Creating images for triangle with 50 training images and 10 validation images


100%|██████████| 10/10 [00:14<00:00,  1.44s/it]


Creating images for circle with 50 training images and 10 validation images


100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


Creating images for star with 50 training images and 10 validation images


100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


In [69]:
create_set_of_images("circle", 50, 20, 40, 120, img_dir_tag="v3_", jitter=True)

Creating images for circle with 50 training images and 20 validation images


100%|██████████| 20/20 [00:16<00:00,  1.24it/s]


In [70]:
create_set_of_images("star", 50, 10, 80, 160)

Creating images for star with 50 training images and 10 validation images


100%|██████████| 10/10 [00:09<00:00,  1.04it/s]
